In [ ]:
# !pip install sentence-transformers
# !pip install datasets
# !pip install evaluate

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 21 01:41:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train_raw.csv to train_raw.csv


In [ ]:
!rm requirements.txt

In [4]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import  AutoModelForSequenceClassification, AutoTokenizer


In [20]:
classifier = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2) # Get the classifier
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### **Training Data Readin**

In [10]:
# Reading Training Data - contains 50K troll & non-troll tweets

train_data_readin = pd.read_csv('train_raw.csv')
train_data_readin.head(5)

,tweetid,user_display_name,tweet_text,clean_text,troll_or_not
0,1245883557362282497,85c9M6CDZxgBwoEye0rF12ZBgGl3xvz6Bnbvhp7MUKI=,"having each tiny wish come true, or having som...",having each tiny wish come true or having some...,1
1,961577921461866496,曲剑明,＠null It is 12:25 UTC now,null It is UTC now,1
2,941616158075211776,IFL1E0m0SRX2cdOtuLFV7xKtnBgxagKzNgkuGFvNtvs=,British number two Bedene to switch back to Sl...,British number two Bedene to switch back to Sl...,1
3,850414479976345600,Klausv,kalamitykait Thanks for bearing with us - you ...,kalamitykait Thanks for bearing with us you sh...,1
4,960784360071925760,曲剑明,＠null It is 08:56 CET now,null It is CET now,1


In [11]:
# keeping clean_text and target column, which is troll_or_not
train_data_readin = train_data_readin.loc[:,["clean_text", "troll_or_not"]]

In [12]:
train_data_readin.head(5)

,clean_text,troll_or_not
0,having each tiny wish come true or having some...,1
1,null It is UTC now,1
2,British number two Bedene to switch back to Sl...,1
3,kalamitykait Thanks for bearing with us you sh...,1
4,null It is CET now,1


### **Creating Training & Validation Set**

In [13]:
train_data, val_data = train_test_split(train_data_readin, train_size=0.8,stratify=train_data_readin.troll_or_not, random_state=1234)

In [14]:
print("training data : ",train_data.shape)
print("validation data : ",val_data.shape)

training data :  (71958, 2)
validation data :  (17990, 2)


In [15]:
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "eval": Dataset.from_pandas(val_data)})

In [16]:
print("Dataset Dict:\n", raw_datasets)
print("\n\nTrain's features:\n", raw_datasets["train"].features)
print("\n\nFirst row of Train:\n", raw_datasets["train"][0])

Dataset Dict:
 DatasetDict({
    train: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__'],
        num_rows: 71958
    })
    eval: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__'],
        num_rows: 17990
    })
})


Train's features:
 {'clean_text': Value(dtype='string', id=None), 'troll_or_not': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None)}


First row of Train:
 {'clean_text': 'Welcome to the age of Hygiene Theater', 'troll_or_not': 0, '__index_level_0__': 45945}


In [21]:
tokenized_datasets = raw_datasets.map(lambda dataset: tokenizer(dataset['clean_text'], truncation=True), batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/71958 [00:00<?, ? examples/s]

Map:   0%|          | 0/17990 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 71958
    })
    eval: Dataset({
        features: ['clean_text', 'troll_or_not', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 17990
    })
})


In [22]:
#validating for completeness
print(tokenized_datasets["train"][0])

{'clean_text': 'Welcome to the age of Hygiene Theater', 'troll_or_not': 0, '__index_level_0__': 45945, 'input_ids': [101, 6160, 2000, 1996, 2287, 1997, 19548, 4258, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [23]:
tokenized_datasets = tokenized_datasets.remove_columns(["clean_text", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("troll_or_not", "labels")
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 71958
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 17990
    })
})


### **Fine Tuning distilBERT - Main Section**

In [57]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args
training_args = TrainingArguments("test-trainer", num_train_epochs=3, evaluation_strategy="epoch",
                                  weight_decay=5e-4, save_strategy="no", report_to="none")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [58]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  accuracy = evaluate.load("accuracy")
  precision = evaluate.load("precision")
  recall = evaluate.load("recall")
  f1 = evaluate.load("f1")

  return {
      "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
      "precision": precision.compute(predictions=predictions, references=labels, average='binary')["precision"],
      "recall": recall.compute(predictions=predictions, references=labels, average='binary')["recall"],
      "f1": f1.compute(predictions=predictions, references=labels, average='binary')["f1"],
  }

In [59]:
trainer = Trainer(
    classifier,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-59-758c9506e17b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.242000,0.295840,0.914619,0.921207,0.906653,0.913872
2,0.082200,0.541138,0.914675,0.922362,0.905429,0.913817
3,0.019800,0.626438,0.917454,0.918928,0.915554,0.917238


TrainOutput(global_step=26985, training_loss=0.07093088864970919, metrics={'train_runtime': 645.3828, 'train_samples_per_second': 334.49, 'train_steps_per_second': 41.812, 'total_flos': 1525021612913232.0, 'train_loss': 0.07093088864970919, 'epoch': 3.0})

In [61]:
# Make prediction on evaluation dataset
y_pred = trainer.predict(tokenized_datasets["eval"]).predictions
y_pred = np.argmax(y_pred, axis=-1)

# Get the true labels
y_true = tokenized_datasets["eval"]["labels"]
y_true = np.array(y_true)

# Print the classification report
print(classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.916     0.919     0.918      9002
           1      0.919     0.916     0.917      8988

    accuracy                          0.917     17990
   macro avg      0.917     0.917     0.917     17990
weighted avg      0.917     0.917     0.917     17990



In [62]:
trainer.save_model('/content/drive/MyDrive/TrollDetection-DistilBERT')

### **Test Data Performance Checks**

In [49]:
uploaded = files.upload()

Saving validate.csv to validate.csv


In [52]:
# Reading Test Data - contains 10K troll & non-troll tweets

test_data_readin = pd.read_csv('validate.csv')
print(test_data_readin.shape)
test_data_readin.head(5)

(10000, 5)


,tweetid,user_display_name,tweet_text,clean_text,troll_or_not
0,1189300879905755142,AbacusNews,"Hungry? With a tap on your phone, you can summ...",Hungry With a tap on your phone you can summon...,0
1,1301520005822058496,guardian,US jobless claims drop sharply as government c...,US jobless claims drop sharply as government c...,0
2,359158016613949441,ZTUhI030WiV97FLuXgAU427wsJ1TtqtTBMXjKgShnU=,IT GOES SOMETHING LIKE THIS,IT GOES SOMETHING LIKE THIS,1
3,1278149046973001729,FrankLuntz,"In Arizona, coronavirus isn’t just a pandemic....",In Arizona coronavirus isnt just a pandemic It...,0
4,1301895307098230785,FoxFriendsFirst,‘100 Days Of Liberal Lawlessness’: Attorneys G...,Days Of Liberal Lawlessness Attorneys General ...,0


In [63]:
test_data_readin = test_data_readin.loc[:,["clean_text", "troll_or_not"]]

test_df = Dataset.from_pandas(test_data_readin)
test_df = test_df.map(lambda dataset: tokenizer(dataset['clean_text'], truncation=True), batched=True)

predictions = trainer.predict(test_df)
preds = np.argmax(predictions.predictions, axis=-1)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [64]:
# unseen data performance
print(classification_report(test_data_readin['troll_or_not'], preds, digits=3))

              precision    recall  f1-score   support

           0      0.916     0.923     0.920      4939
           1      0.925     0.917     0.921      5061

    accuracy                          0.920     10000
   macro avg      0.920     0.920     0.920     10000
weighted avg      0.920     0.920     0.920     10000

